## 01. Load mgatk variants (and filter if needed)

In [1]:
# indir = "/home/isshamie/data/Projects/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor0"
# name = "donor0"
# donor = 0
# outdir = "/home/isshamie/data/Projects/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/donor0"
#kparam = 10 

# indir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/subsamples/pct_75/0/donor1"
# name = "donor1" 
# donor = 1 
# outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/subsamples/pct_75/0/donor1/knn/kparam_30"
# kparam = 30

# Cluster parameters
cells_col <- "donor"
cell_id_col <- "ID"


indir = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/subsamples/pct_10/81/donor3"
name = "donor3"
donor = 3
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/subsamples/pct_10/81/donor3/knn/kparam_50"
kparam = 50

algorithm = 1
# (1 = original Louvain algorithm; 2 = Louvain algorithm with multilevel refinement; 3 = SLM algorithm; 4 = Leiden algorithm). Leiden requires the leidenalg python.


### Load Libraries

In [2]:
library(data.table)
library(dplyr)
library(SummarizedExperiment)
library(Matrix)
library(BuenColors)

library(future)
plan()
#plan("multiprocess", workers = workers)
options(future.globals.maxSize = 8000 * 1024^2)

library(Seurat)
library(Signac)

Warning message:
“package ‘dplyr’ was built under R version 4.1.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MatrixGenerics

Loading required package: matrixStats

Warning message:
“package ‘matrixStats’ was built under R version 4.1.3”

Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Registered S3 method overwritten by 'spatstat.core':
  method          from
  formula.glmmPQL MASS

Attaching SeuratObject

Attaching sp


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays




In [3]:
# SEall <- readRDS(indir) #"/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/pre/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/pre.signac.rds")
# SEall

## Subset on cells and variants if given

In [4]:
af_ind = file.path(indir, "af.tsv")

In [5]:
af_ind

[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/subsamples/pct_10/81/donor3/af.tsv"

In [6]:
af.vals = t(read.table(af_ind,header=T, row.names=1, check.names=F))
print(head(af.vals))


       TAAGTGCAGTTACACC-1_Input AGTCAACAGTGAGTGC-1_Input
5214A                         0              0.000000000
263G                          1              1.000000000
15113T                        0              0.000000000
150T                          0              0.000000000
3655A                         0              0.006802721
4769G                         1              1.000000000
       CTAGGATCAAGCACTT-1_Input ACCCAAAGTCCTTCAC-1_Input
5214A                         0                        0
263G                          1                        1
15113T                        0                        0
150T                          0                        0
3655A                         0                        0
4769G                         1                        1
       AAATGAGTCTCTGCAC-1_Input AAATGCCTCGTGGTAT-1_Input
5214A                         0                        0
263G                          1                        1
15113T                        0

In [7]:
#af.vals

In [8]:
cell.meta <- data.frame(row.names=colnames(af.vals), condition=gsub("^.*_", "",colnames(af.vals)),
                       stringsAsFactors=T)
cell.meta

,condition
,<fct>
TAAGTGCAGTTACACC-1_Input,Input
AGTCAACAGTGAGTGC-1_Input,Input
CTAGGATCAAGCACTT-1_Input,Input
ACCCAAAGTCCTTCAC-1_Input,Input
AAATGAGTCTCTGCAC-1_Input,Input
AAATGCCTCGTGGTAT-1_Input,Input
GGAGAACGTTAACCAC-1_Input,Input
TCCCACACACTCCCAT-1_Input,Input
TGGGTTAGTGTGACCC-1_Input,Input


In [9]:
# nrows <- 200
# ncols <- 6
# counts <- matrix(runif(nrows * ncols, 1, 1e4), nrows)
# rowRanges <- GRanges(rep(c("chr1", "chr2"), c(50, 150)),
#                      IRanges(floor(runif(200, 1e5, 1e6)), width=100),
#                      strand=sample(c("+", "-"), 200, TRUE),
#                      feature_id=sprintf("ID%03d", 1:200))
# colData <- DataFrame(Treatment=rep(c("ChIP", "Input"), 3),
#                      row.names=LETTERS[1:6])
# colData

# assay(SummarizedExperiment(assays=list(counts=counts),
#                      rowRanges=rowRanges, colData=colData))

## 02. Cluster using Seurat FindNeighbors (S-NN graph ) and FindClusters (louvain)

In [10]:
getNN <- function(mat_af, k.param = 10){
  set.seed(1)
  rownames(mat_af) <- make.unique(rownames(mat_af))
  obj <- FindNeighbors(mat_af, k.param = k.param, annoy.metric = "cosine")
  obj
}

# Function to use Seurat functions to call mitochondrial cluster clones
seuratSNN_cosineDistance <- function(obj, resolution){
  clusters <- FindClusters(object = obj$snn, resolution = resolution, algorithm=algorithm)
  return(as.character(clusters[,1]))
}


In [11]:
plot_af_clusters <- function(afin, cluster_name){
    mdf <- data.frame(cluster_name, t(afin), check.names=F) %>%
    group_by(cluster_name) %>% dplyr::filter(n() >= 5) %>% summarize_all(.funs = mean)

    melt_df <- reshape2::melt(mdf, id.vars = "cluster_name")

    # This series of arrangements is for aesthetics
    melt_df2 <- melt_df %>% mutate(g5 = value >= 0.05, g1 = value >= 0.01, sqrt = sqrt(value)) %>%  group_by(variable) %>%
    mutate(total_g1 = sum(g1)) %>% mutate(total_g5 = sum(g5)) %>% 
    arrange(desc((g5)), desc((g1))) %>% dplyr::filter(total_g1 > 0) 

    # More aesthetics
    melt_df2$cluster_name <- factor(as.character(melt_df2$cluster_name), levels = rev(unique(as.character(melt_df2$cluster_name))))
    melt_df2$variable <- factor(as.character(melt_df2$variable ), levels = (unique( as.character(melt_df2$variable))))
    melt_df2$value <- ifelse(melt_df2$value > 0.05, 1, melt_df2$value)
    melt_df2$value <- ifelse(melt_df2$value < 0.005, 0, melt_df2$value)
    

    # Visualize heatmap of clusters x variants number cells > 0.01
    p.numvar1 <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = total_g5)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.title.y=element_blank(),
          axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ggtitle("Total number of cells greater than 0.05 in variant") +
    L_border() #+
    #theme(legend.position = "none")
    ggsave(file.path(outdir, paste0(name,".variants.g5.png")))
    #ggsave(file.path(outdir, "variants.png"))
    #cowplot::ggsave(p1, file = paste0("../plots/mutations_clones_",what,"_invivo_grid.pdf"), width = 3.2, height = 1.5)
    print(length(unique(melt_df2$variable)))
    print(length(unique(melt_df2$cluster_name)))

    # Visualize heatmap of clusters x variants
    p.bin <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = value)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ylab("Cluster") +
    ggtitle("Variant avg greater than 0.05 set to 1 and <0.005 to 0") +
    L_border() #+
           
    #theme(legend.position = "none")
    print("Number of variants")
    print(length(unique(melt_df2$variable)))
    print("Number of clusters")
    print(length(unique(melt_df2$cluster_name)))
    ggsave(file.path(outdir, paste0(name, ".variants.binary.png")))
    
    p.varmean <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = sqrt)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.title.y=element_text("Cluster"),
          axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ylab("Cluster") +
    ggtitle("Sqrt Mean AF in each cluster") +
    L_border() #+
    ggsave(file.path(outdir, paste0(name,".variants.labels.png")))
    #theme(legend.position = "none")

    return(c(melt_df2, p.numvar1, p.varmean))
}

In [15]:
run_clones <- function(SE, curr_out, k.param, resolution=3.5){
    afin <- data.matrix(assay(SE))  
    #print('afin')
    #print(head(afin))
    #print(head(rownames(afin)))
    # Since the PBMCs alone undercalled the mutations, we will examine the mutations called in the PBMCs and CD34s together
    obj <- getNN(t(sqrt(afin)), k.param=k.param)
    #print(obj)
    tryCatch({ 
        clusters <- seuratSNN_cosineDistance(obj, resolution = resolution) 
    },error=function(cond){
        print('here')
        clusters <- seuratSNN_cosineDistance(obj, resolution = 2) 
        #print('clusters')
        #print(clusters)
    })
    cluster_name <- stringr::str_pad(as.character(clusters), 3, pad = "0")
    print(table(cluster_name))
    saveRDS(clusters, file.path(curr_out, "clusters.rds"))
    saveRDS(obj, file.path(curr_out, "af.dimRed.rds"))
    #print(head(stringr::str_split(colnames(afin), "_")[]))
    condition <- stringr::str_split(colnames(afin), "_", simplify=T)[,2]
    print('condition')
    print(head(condition))
    cell_clusters <- data.frame(ID=colnames(afin), lineage=cluster_name, donor=donor, condition=condition)
    write.table(cell_clusters, file.path(curr_out, "cells_meta.tsv"), row.names=F, sep="\t", quote=F)
    out <- plot_af_clusters(afin, cluster_name)
    melt_df2 <- out[[1]]
    write.table(melt_df2, file.path(curr_out, "var_clusters.tsv"), row.names=F, sep="\t", quote=F)
    return(out)
}

# Run workflow

In [13]:
SE <- SummarizedExperiment(assays=list(allele_frquency=af.vals), colData = cell.meta)
print(dim(assay(SE)))



[1] 255  94


In [16]:

#colnames(SE) <- lapply(colnames(SE), function(x) paste0(x, name))
print("clones w all cells")
#curr_SE <- SE #filt_vars(SE, cells_f, vars_f)
out <- run_clones(SE, outdir, k.param=kparam)

[1] "clones w all cells"


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 94
Number of edges: 4371

Running Louvain algorithm...
Maximum modularity in 10 random starts: -0.0381
Number of communities: 94
Elapsed time: 0 seconds


Warning message in max(connectivity, na.rm = T):
“no non-missing arguments to max; returning -Inf”


[1] "here"
Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 94
Number of edges: 4371

Running Louvain algorithm...
Maximum modularity in 10 random starts: -0.0215
Number of communities: 81
Elapsed time: 0 seconds


74 singletons identified. 7 final clusters.



ERROR: Error in stri_pad_left(string, width, pad = pad): object 'clusters' not found


In [ ]:
outdir